In [ ]:
import os

import cPickle as pickle
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import stopwords
from gensim.similarities import SoftCosineSimilarity
from gensim.corpora import Dictionary

from gensim.models.doc2vec import TaggedLineDocument
import time
import logging
import argparse
import numpy as np
import multiprocessing
from sklearn.decomposition import PCA
from matplotlib import pyplot
from gensim.parsing.preprocessing import remove_stopwords

from gensim.models import Phrases
from gensim.models.phrases import Phraser

from gensim.similarities import WmdSimilarity

import sys
import codecs

In [ ]:
data = LineSentence('countries_filter.txt')

In [ ]:
contents = TaggedLineDocument("countries_filter.txt")

In [ ]:
domain_vocab_file = "Sports Sport sport players teams team goal score scores scored"
vocab_list = domain_vocab_file.split()


dim = 200
win = 12
neg = 5

In [ ]:
cores = multiprocessing.cpu_count()

model = Doc2Vec(contents, vector_size=dim, window=win, 
                    min_count=1, workers=cores,hs=0,negative=5,
                    dm=0,dbow_words=1,epochs=20, smoothing=0.5,
                    sampling_param=0.7, objective_param=0.5, vocab_file=vocab_list)

In [ ]:
for d in contents:
    print d[0]

In [ ]:
print model.docvecs[40]


In [ ]:
print model.docvecs[39]

In [ ]:
from scipy import spatial

In [ ]:
results2 = 1 - spatial.distance.cosine(model.docvecs[37], model.docvecs[0])

In [ ]:
print results2

In [ ]:
print model.similar_by_vector(model.docvecs[0],topn=10)

In [ ]:
from gensim.test.utils import common_texts

In [ ]:
all_docs = []
for d in data:
    all_docs.append(d)

In [ ]:
print all_docs[0]

In [ ]:
for doc in contents:
            print doc[0][0:4]
            inferred_docvec = model.infer_vector(doc.words)
            print model.wv.most_similar([inferred_docvec], topn=10)

In [ ]:
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
   pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show() 

In [ ]:
dictionary = Dictionary(all_docs)

In [ ]:
bow_corpus = [dictionary.doc2bow(document) for document in all_docs]

In [ ]:
similarity_matrix = model.wv.similarity_matrix(dictionary)

In [ ]:
index = SoftCosineSimilarity(bow_corpus, similarity_matrix, num_best=10)

In [ ]:
query = 'football is a beautiful sport, i like it the most among all Sports'.split()
sims = index[dictionary.doc2bow(query)]

In [ ]:
print sims

In [ ]:
print model.similar_by_word("Sports", topn=10)

In [ ]:
model.init_sims(replace=True)

In [ ]:
distance = model.wmdistance("sport", "sport") 
print distance

In [ ]:
num_best = 40
instance = WmdSimilarity(all_docs, model, num_best=40)

In [ ]:
sent = 'football is a beautiful sport, i like it the most among all Sports.'.split()

sims = instance[sent] 

In [ ]:
print 'Query:'
print sent
for i in range(num_best):
    print
    print 'sim = %.4f' % sims[i][1]
    print all_docs[sims[i][0]]